![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Mathematics/FlippingCoins/flipping-coins.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Flipping Lots of Coins

A classic statistics experiment is simply counting how many "heads" and "tails" you observe when flipping a coin repeatedly. With a perfectly unbiased coin in a statistically perfect world, we expect to count an equal number of heads and tails.

<img src="images/iStock-coinflip.jpg" width="400px">

However, in the real world there are **statistical fluctuations**, meaning that experimental data are not always equal to what you would expect. For example, if you were to flip a coin 100 times, you might observe 52 heads and 48 tails or even 45 heads and 55 tails. Of course the more times you flip a coin and record the outcome, the closer you are likely to get to a 50:50 ratio.

This sort of statistical problem where you run a number of trials is easily simulated using Python with a **random number generator**.

Select the code cell below, then click the `▶Run` button to `print` 10 random numbers between 0 and 1.

In [ ]:
import random
for i in range(10):
    print(random.uniform(0,1))

If you run the above cell a few times, you should find that the numbers are different each time.

To simulate a random coin flip, though, we can use `random.choice` to choose between `heads` and `tails`.

In [ ]:
for i in range(10):
    print(random.choice(['heads', 'tails']))

Now that we know how to simulate a coin flip, let's flip a lot of coins.

In [ ]:
N = 100
results = [random.choice(['heads', 'tails']) for i in range(N)]

import pandas as pd
df = pd.DataFrame(results, columns=['Result'])
df

Let's visualize the results of that set of coin flips.

In [ ]:
import plotly.express as px
px.histogram(df, x='Result', title='Results of {} Coin Flips'.format(N))

We can also animate the counting of the coin flips. This might require a lot of calculating if there were more than 100 coin flips.

In [ ]:
df['Heads'] = (df['Result']=='heads').cumsum()
df['Tails'] = (df['Result']=='tails').cumsum()
df['Flip'] = df.index
adf = df.drop('Result', axis=1).melt(id_vars='Flip', var_name='Result', value_vars=['Heads', 'Tails'], value_name='Count')
ymax = adf['Count'].max()
fig = px.histogram(adf, x='Result', y='Count', animation_frame='Flip', title='Flipping {} Coins'.format(N))
fig.update_xaxes(range=[-0.5,1.5]).update_yaxes(title='Count', range=[0,ymax])
fig.show()

Let's simulate flipping more coins and graphing, without animation, the results of those coin flips. You can try changing `N = 100000` to even larger numbers if you'd like.

In [ ]:
N = 100000

#px.histogram(pd.DataFrame([random.choice(['heads', 'tails']) for i in range(N)], columns=['Result']), x='Result', title='Results of {} Coin Flips'.format(N))
px.histogram([random.choice(['heads', 'tails']) for i in range(N)], title='Results of {} Coin Flips'.format(N)).update_xaxes(title='Result').update_layout(showlegend=False)

We see that the more coin flips we observe, the more likely we are to get close to the theoretical $1:1$ ratio.

## Biased Coin

So far we have been assuming that our coin is equally likely to land on heads or tails. We could also model a *biased* coin that is more likely to land on one than the other. For example, suppose we have a coin with a 60% probability to land with heads facing up.

To simulate this, we specify the probability then draw a random number. If that random number is less than the probability of heads we return heads, if it's greater we return tails. This process as a flowchart would appear as follows.

![alt text](images/coin_flip_ex.png)

In [ ]:
N = 100000
probability_of_heads = 0.6

results = ['heads' if random.uniform(0,1)<probability_of_heads else 'tails' for i in range(N)]

px.histogram(results, title='Results of {} Coin Flips with a Probability of Heads of {}'.format(N, probability_of_heads)).update_xaxes(title='Result').update_layout(showlegend=False)

Feel free to run the above code cell as many times as you like to simulate coin tosses. You can change both `N = 100000` and `probability_of_heads = 0.6` to see how they affect the graph.

# Conclusion

In this notebook we covered the idea of statistical fluctuations and how we can use the random number generator to model those statistical fluctuations. In doing so, we created a simulation of coin tosses for both fair and biased coins and observed the difference in counting simulated experimental statistics between the two. This sort of model can be applied to nearly any statistical process, and can act as a primer for the idea of statistical simulations and [Monte Carlo methods](https://en.wikipedia.org/wiki/Monte_Carlo_method).

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)